## Part I

In [1]:
#1. Import the standard libraries and others as needed.

from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd
import numpy as np
import datetime

In [2]:
#2. Connect to MongoDB.
#3. Restore the database from the dump given in the ‘data.zip’ file.

client = MongoClient('localhost', 27017)
print(client.list_database_names())
print(client.patient_db.list_collection_names())

['admin', 'config', 'local', 'patient_db']
['conditions', 'patients']


In [3]:
#4. Import the ‘patient_data_1.csv’ and ‘patient_data_2.csv’ into pandas DataFrames and harmonize them.
#(1) patient_data_1.csv

data1 = pd.read_csv("patient_data_1.csv")
data1.rename(columns={'Id': '_id'}, inplace=True) #change 'Id' to '_id'
data1['GENDER'].replace('Male', 'M', inplace=True) #change 'Male' to 'M'
data1['GENDER'].replace('Female', 'F', inplace=True) #change 'Female' to 'F'
data1

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P004753,2012-01-12,NaN,asian,hispanic,F,Westford,Middlesex County,Massachusetts
1,P009495,1961-12-01,M,white,nonhispanic,F,Seekonk,Bristol County,Massachusetts
2,P010851,1997-10-21,NaN,black,nonhispanic,M,Quincy,Norfolk County,Massachusetts
3,P009020,1975-08-15,M,white,nonhispanic,F,Worcester,Worcester County,Massachusetts
4,P009779,2020-02-25,NaN,white,nonhispanic,F,Bolton,Worcester County,Massachusetts
...,...,...,...,...,...,...,...,...,...
838,P009210,2003-09-13,NaN,white,nonhispanic,F,Boston,Suffolk County,Massachusetts
839,P009845,2003-11-28,NaN,white,nonhispanic,F,Medford,Middlesex County,Massachusetts
840,P001189,1985-01-10,M,white,nonhispanic,F,Worcester,Worcester County,Massachusetts
841,P002360,1970-09-15,M,white,nonhispanic,M,Newton,Middlesex County,Massachusetts


In [4]:
#(2) patient_data_2.csv

data2 = pd.read_csv("patient_data_2.csv")
data2.columns = data2.columns.map(lambda x:x.upper()) #change column names to upper case
data2.rename(columns={'ID': '_id'}, inplace=True) #change 'ID' to '_id'
data2['ETHNICITY'].replace('non-hispanic', 'nonhispanic', inplace=True) #change 'non-hispanic' to 'nonhispanic'
data2

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P005102,1969-05-11,M,white,nonhispanic,F,East Longmeadow,Hampden County,Massachusetts
1,P001814,1957-12-31,M,white,nonhispanic,M,Somerville,Middlesex County,Massachusetts
2,P000373,1989-07-20,S,white,nonhispanic,F,Cambridge,Middlesex County,Massachusetts
3,P007365,1967-10-26,M,white,nonhispanic,F,Mashpee,Barnstable County,Massachusetts
4,P010311,1965-02-14,M,white,nonhispanic,F,Somerville,Middlesex County,Massachusetts
...,...,...,...,...,...,...,...,...,...
452,P005128,1962-10-21,M,white,nonhispanic,M,Boston,Suffolk County,Massachusetts
453,P010979,2000-03-02,NaN,asian,nonhispanic,M,Dracut,Middlesex County,Massachusetts
454,P003574,1978-05-22,M,white,nonhispanic,F,Marshfield,Plymouth County,Massachusetts
455,P007576,2008-11-24,NaN,white,nonhispanic,F,Shrewsbury,Worcester County,Massachusetts


In [5]:
#5. From the ‘patients collection, retrieve the patients who are alive. 
#(1) Print the number of patients retrieved.

pts = client.patient_db.patients
pts.count_documents({'DEATHDATE':{'$exists':False}})

8700

In [6]:
#(2) Store them in a DataFrame.

data3 = pd.DataFrame(pts.find({'DEATHDATE':{'$exists':False}}))
data3

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts
...,...,...,...,...,...,...,...,...,...
8695,P012343,1960-01-05,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts
8696,P012345,1980-11-12,M,white,nonhispanic,F,Malden,Middlesex County,Massachusetts
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts


In [7]:
#6. Concatenate the above three DataFrames.

cond = pd.concat([data1, data2, data3])
cond = cond.sort_values(by = '_id')
cond

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts
...,...,...,...,...,...,...,...,...,...
591,P012346,1990-08-25,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts
249,P012348,1962-08-17,M,asian,nonhispanic,F,Wellesley,Norfolk County,Massachusetts
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts


In [8]:
#7. Compute the current age of the patients and store them in another column called ‘AGE’. 

cond['BIRTHDATE'] = pd.to_datetime(cond['BIRTHDATE'])
today = datetime.datetime.today().year #this year

cond['AGE'] = today - cond['BIRTHDATE'].dt.year
cond

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE,AGE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,6
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts,7
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts,31
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts,19
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts,27
...,...,...,...,...,...,...,...,...,...,...
591,P012346,1990-08-25,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts,33
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts,63
249,P012348,1962-08-17,M,asian,nonhispanic,F,Wellesley,Norfolk County,Massachusetts,61
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts,105


In [9]:
#8. Insert the above DataFrame to a collection named ‘patients_clean’.

client.patient_db.patients_clean.insert_many(cond.to_dict('records'))
print(client.patient_db.list_collection_names())
print(client.patient_db.patients_clean.count())

['patients_clean', 'conditions', 'patients']
10000


<ipython-input-9-f74d20957fe7>:5: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print(client.patient_db.patients_clean.count())


## Part II

In [2]:
client = MongoClient('localhost', 27017)
print(client.list_database_names())
print(client.patient_db.list_collection_names())

['admin', 'config', 'local', 'patient_db']
['patients_clean', 'conditions', 'patients']


In [3]:
#9. Obtain the ID’s of patients that were diagnosed with COVID-19
#(1) 8820 patients diagnosed with COVID-19

cdt = client.patient_db.conditions
cdt.count_documents({'conditions': 'COVID-19'})

8820

In [4]:
#(2) ID’s of COVID-19 patients

list(cdt.find({'conditions': 'COVID-19'}, {'_id': 0, 'patient': 1}))

[{'patient': 'P000001'},
 {'patient': 'P000002'},
 {'patient': 'P000003'},
 {'patient': 'P000004'},
 {'patient': 'P000005'},
 {'patient': 'P000006'},
 {'patient': 'P000007'},
 {'patient': 'P000008'},
 {'patient': 'P000009'},
 {'patient': 'P000011'},
 {'patient': 'P000013'},
 {'patient': 'P000014'},
 {'patient': 'P000015'},
 {'patient': 'P000016'},
 {'patient': 'P000017'},
 {'patient': 'P000018'},
 {'patient': 'P000020'},
 {'patient': 'P000021'},
 {'patient': 'P000022'},
 {'patient': 'P000023'},
 {'patient': 'P000024'},
 {'patient': 'P000025'},
 {'patient': 'P000026'},
 {'patient': 'P000027'},
 {'patient': 'P000028'},
 {'patient': 'P000030'},
 {'patient': 'P000031'},
 {'patient': 'P000034'},
 {'patient': 'P000035'},
 {'patient': 'P000036'},
 {'patient': 'P000037'},
 {'patient': 'P000038'},
 {'patient': 'P000039'},
 {'patient': 'P000040'},
 {'patient': 'P000041'},
 {'patient': 'P000042'},
 {'patient': 'P000043'},
 {'patient': 'P000044'},
 {'patient': 'P000045'},
 {'patient': 'P000047'},


In [5]:
#10. Create an aggregation pipeline to obtain COVID-19 patients who also had ‘Cough (finding)’ and ‘Fever (finding)’. 
#(1) 5334 patients included

cdt.count_documents({'$and': 
                     [{'conditions': 'COVID-19'}, 
                      {'conditions': 'Cough (finding)'}, 
                      {'conditions': 'Fever (finding)'}]
                    })

5334

In [6]:
#(2) the list of these patients

list(cdt.find({'$and': [{'conditions': 'COVID-19'}, {'conditions': 'Cough (finding)'}, {'conditions': 'Fever (finding)'}]}))

[{'_id': 'E000005',
  'patient': 'P000002',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Diarrhea symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000009',
  'patient': 'P000003',
  'conditions': ['Headache (finding)',
   'Cough (finding)',
   'Fatigue (finding)',
   'Nausea (finding)',
   'Vomiting symptom (finding)',
   'Fever (finding)',
   'Loss of taste (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000010',
  'patient': 'P000004',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Nausea (finding)',
   'Vomiting symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000011',
  'patient': 'P000005',
  'conditions': ['Nasal congestion (finding)',
   'Cough (finding)',
   'Sputum finding (finding)',
   'Fatigue (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000028',
  'patient': 'P000011',
  'con

In [7]:
#11. Create a new aggregation pipeline to join patient data for 8,820 COVID-19 patients. 
#(1) 8463 alive COVID-19 patients

len(list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        }
    ]
)))

8463

In [8]:
#(2) the first 5 patients

list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        },
        {
            '$limit': 5
        }
    ]
))

[{'_id': 'E000003',
  'patient': 'P000001',
  'conditions': ['Fever (finding)', 'Suspected COVID-19', 'COVID-19'],
  'detail': [{'_id': 'P000001',
    'BIRTHDATE': datetime.datetime(2017, 8, 24, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'M',
    'CITY': 'Springfield',
    'COUNTY': 'Hampden County',
    'STATE': 'Massachusetts',
    'AGE': 6}]},
 {'_id': 'E000005',
  'patient': 'P000002',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Diarrhea symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19'],
  'detail': [{'_id': 'P000002',
    'BIRTHDATE': datetime.datetime(2016, 8, 1, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'F',
    'CITY': 'Walpole',
    'COUNTY': 'Norfolk County',
    'STATE': 'Massachusetts',
    'AGE': 7}]},
 {'_id': 'E000009',
  'patient': 'P000003',
  'conditions': ['Headache (finding)',
   'Cough (finding)',
   'Fati

In [9]:
#12. Based on the previous pipeline, create a new pipeline to calculate the total number of 
#alive COVID-19 patients for each gender.
#(1) 3992 Males

len(list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        },
        {
            '$match': {
                'detail.GENDER': 'M'
            }
        }
    ]
)))

3992

In [10]:
#the first 5 male patients

list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        },
        {
            '$match': {
                'detail.GENDER': 'M'
            }
        },
        {
            '$limit': 5
        }
    ]
))

[{'_id': 'E000003',
  'patient': 'P000001',
  'conditions': ['Fever (finding)', 'Suspected COVID-19', 'COVID-19'],
  'detail': [{'_id': 'P000001',
    'BIRTHDATE': datetime.datetime(2017, 8, 24, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'M',
    'CITY': 'Springfield',
    'COUNTY': 'Hampden County',
    'STATE': 'Massachusetts',
    'AGE': 6}]},
 {'_id': 'E000009',
  'patient': 'P000003',
  'conditions': ['Headache (finding)',
   'Cough (finding)',
   'Fatigue (finding)',
   'Nausea (finding)',
   'Vomiting symptom (finding)',
   'Fever (finding)',
   'Loss of taste (finding)',
   'Suspected COVID-19',
   'COVID-19'],
  'detail': [{'_id': 'P000003',
    'BIRTHDATE': datetime.datetime(1992, 6, 30, 0, 0),
    'MARITAL': 'S',
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'M',
    'CITY': 'Chicopee',
    'COUNTY': 'Hampden County',
    'STATE': 'Massachusetts',
    'AGE': 31}]},
 {'_id': 'E000011',
  'patient': 'P0000

In [11]:
#(2) 4471 Females

len(list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        },
        {
            '$match': {
                'detail.GENDER': 'F'
            }
        }
    ]
)))

4471

In [12]:
#the first 5 female patients

list(cdt.aggregate(
    [
        {
            '$lookup': {
                'from': 'patients_clean', 
                'localField': 'patient',
                'foreignField': '_id',
                'as': 'detail'
            }
        },
        {
            '$match': {
                'conditions': 'COVID-19'
            }
        },
        {
            '$match': {
                'detail': {'$ne':[]}
            }
        },
        {
            '$match': {
                'detail.GENDER': 'F'
            }
        },
        {
            '$limit': 5
        }
    ]
))

[{'_id': 'E000005',
  'patient': 'P000002',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Diarrhea symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19'],
  'detail': [{'_id': 'P000002',
    'BIRTHDATE': datetime.datetime(2016, 8, 1, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'F',
    'CITY': 'Walpole',
    'COUNTY': 'Norfolk County',
    'STATE': 'Massachusetts',
    'AGE': 7}]},
 {'_id': 'E000010',
  'patient': 'P000004',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Nausea (finding)',
   'Vomiting symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19'],
  'detail': [{'_id': 'P000004',
    'BIRTHDATE': datetime.datetime(2004, 1, 9, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'F',
    'CITY': 'Pembroke',
    'COUNTY': 'Plymouth County',
    'STATE': 'Massachusetts',
    'AGE': 19}]},
